# Generation of Level 1 data with romanisim and processing it with romancal. 

***

## Imports
 Libraries used
- *subprocess* for running scripts from command line 
- *numpy* to handle array functions
- *pandas* for working with tabular data
- *matplotlib.pyplot* for plotting data
- *astropy.io* wrting csv files
- *astropy.time* creating a time object 
- *astroquery.gaia* querying and downloading the Gaia data
- *romanisim.gaia* querying and downloading the Gaia data
- *romancal.pipeline* for running the processing pipeline
- *asdf* for reading and writing asdf files
- *roman_datamodels* for reading and writing asdf files

In [ ]:
%matplotlib ipympl
#%matplotlib inline
import os
import sys
import subprocess
import numpy as np
import pandas
import matplotlib.pyplot as plt
from  matplotlib.colors import LogNorm
import astropy.io 
from astroquery.gaia import Gaia
import asdf
import roman_datamodels as rdm
from romancal.pipeline import ExposurePipeline
import romanisim.gaia
import astropy.time 

## Introduction
The purpose of this notebook is to show how to generate simulated Level 1 and Level 2 roman data with ```romanisim``` and then process Level 1 data with ```romancal``` to produce Level 2 data. Additionally, we show  how to open, explore and work with the generated data files.  


Details about the roman data levels can be found [here](https://roman-docs.stsci.edu/data-handbook-home/wfi-data-format/data-levels-and-products). Broadly speaking Level 1 is three-dimensional data cube (one dimension for time and two dimensions for image coorrdinates) that represents a single uncalibrated ramp exposure (unit DN).  More specifically, it is shaped as an array with shape (N resultants, 4096 image rows, 4096 image columns). The Level 2 WFI data are calibrated rate images in instrumental units of (photo)electrons / second (e– / sec). 

The ```romancal``` pipeline also detects sources in the Level 2 images. A list of activities we perform here is as follows. We generate catalogs to be used as input by ```romanisim```, both synthetically and by querying the Gaia archive. Then we run romanisim and ```romancal```.
Next we move on to analysis. We read the source list and view Level 2 image cutouts around the identified sources. We plot ramps from Level 1 corresponding to these identified sources. Finally, we estimate slopes of these ramps and compare the values with those in the Level 2 images.   



***

## Loading data
In this tutorial ```romanisim``` is used to generate simulated data. 
It takes as input a catalog that specifies the list of targets to be simulated. Currently it simulates two types of targets, point sources labelled by type ```PSF``` (e.g stars) and extended sources labelled by ```SER``` (e.g. galaxies).
We begin by defining a utility function to generate a catalog that can be passed on to ```romanisim```.  


### Make input catalog for romanisim synthetically.    
Make a catalog in csv format to be used as input with ```romanisim```, specifying the targets to be simulated.
Targets are distributed uniformly over a rectangular patch (aspect ratio 2:1, minor axis pointing north) 
with user specified center coordinates and area.

In [ ]:
def make_catalog_synthetic(outfile_name, ra, dec, bandpass='F158', n_ser=0, n_psf=0, area=0.24, seed=12):
    """
    Input
    -----
    outfile_name (str): output file name
    ra (float): ra of the center of WFI  [deg]
    dec (float): dec of the center of WFI  [deg]
    bandpass (str): Photometric bandpasss e.g F158
    flux (float): flux in [maggies], such that AB magnitude = -2.5*log(flux) 
    n_sources (int): number of sources to simulate
    area (float): Area over which sources are distributed [square degrees]     
    seed (int): Seed for random number generator
    """
    np.random.seed(seed)
    scale=np.sqrt(area/18.0)
    # total number of sources
    n_sources=n_psf+n_ser
    d={}
    d['ra']=ra+(np.random.uniform(size=n_sources)-0.5)*scale*6/np.cos(np.radians(dec))
    d['dec']=dec+(np.random.uniform(size=n_sources)-0.5)*scale*3    
    # set the type 'PSF' and 'SER' 
    d['type']=np.array(['PSF']*n_psf+['SER']*n_ser)
    # set sersic index of sources in range [2, 4]
    d['n']=np.int64(2+np.random.uniform(size=n_sources)*2)
    # set half light radius radius [3, 8]
    d['half_light_radius']=3+np.random.uniform(size=n_sources)*5
    # set position angle in range [0,90]
    d['pa']=np.random.uniform(size=n_sources)*90.0
    # set major to minor axis ratio in range [0.3,0.7]
    d['ba']=0.3+np.random.uniform(size=n_sources)*0.7
    # set bandpass flux  
    temp1=np.power(10.0,np.random.uniform(size=n_psf))*1e-8
    temp2=np.power(10.0,np.random.uniform(size=n_ser))*1e-7
    d['F158']=np.concatenate([temp1,temp2])
    # write the catalog file
    #pandas.DataFrame(d).to_csv(filename,sep=' ',index=False)
    astropy.io.ascii.write(d,outfile_name,overwrite=True,names=['ra','dec','type','n','half_light_radius','pa','ba','F158'])    
    

dirname='test_data/'
if not os.path.exists(dirname):
    os.makedirs(dirname)

ra_cen, dec_cen = (0.0, 0.0)
make_catalog_synthetic(dirname+'catalog_mixed.csv', ra_cen, dec_cen, n_ser=900,n_psf=360)


### Make input catalog for romanisim from Gaia
We query the Gaia archive to do a cone serach around user specified equatorial coordiantes and then wriyte it to an [```.ecsv```](https://docs.astropy.org/en/stable/io/ascii/ecsv.html) file (a csv file with extra commented out rows at the top providing details about the columns). Flux in roman bandpass is assumed to be 1/100 times (5 magnitudes) fainter than flux in gaia bandpass  ```phot_g_mean_mag```.

In [ ]:

def make_catalog_gaia(outfile_name, ra, dec, date='2026-01-01', bandpasses=['F158'], radius=1.0, row_limit=1000):
    """
    date (str): 'YYYY-MM-DD'  e.g. '2027-06-16' 
    radius (float): search radius in degrees
    row_limit (int): Use -1 for unlimited
    bandpasses list(str): list of roman bandpasses. If set to None the full list is loaded as specified in. 
    """
    row_limit = f'TOP {row_limit}' if row_limit > -1 else ''    
    q = f'select {row_limit} * from gaiadr3.gaia_source where distance({ra}, {dec}, ra, dec) < {radius}'
    job = Gaia.launch_job_async(q)
    r = job.get_results()
    #print(job)
    r['phot_g_mean_mag']=r['phot_g_mean_mag']+5   
    if bandpasses is None:
        bandpasses=set(romanisim.bandpass.galsim2roman_bandpass.values())

    if outfile_name.endswith('.ecsv'):
        romanisim.gaia.gaia2romanisimcat(r, astropy.time.Time(date), fluxfields=set(bandpasses)).write(outfile_name, overwrite=True)
    else:
        raise RuntimeError('output file name should end with .ecsv')
        
dirname='test_data/'
make_catalog_gaia(f"{dirname}catalog_gaia.ecsv",ra_cen, dec_cen)

### Generate L1 and L2 data with romanisim
We use the command line script ```romanisim-make-image``` to generate the simulated data. One can pass a number of arguments to the script. You can checkout the usage by running the cell below or by accessing the [romanisim documentation](https://romanisim.readthedocs.io/en/latest/index.html).

In [ ]:

print(subprocess.getoutput('romanisim-make-image -h'))

We simulate data for detector 2 and pass on the name of the input catalog along with other arguments as required, e.g. output filename, data level and so on. By convention Level 1 data have suffix ```_uncal```. To generate the level 2 data corresponding to a given Level 1 data, we need to make sure that the  same seed is used for the random number generator (```rng_seed```).

In [ ]:
sca=2
exposure_no=1
date='2026-01-01T01:00:00'
dirname='test_data/'
l1_fname=f"romanisim_mixed_sca{sca}"
catalog_filename=dirname+'catalog_mixed.csv'

level=1
filename=f"{dirname}{l1_fname}_uncal.asdf"        
cmd = f"romanisim-make-image --radec {ra_cen} {dec_cen} --catalog {catalog_filename} --usecrds --webbpsf  --level {level} --bandpass F158 --date {date} --rng_seed 11 --sca {sca} {filename}"
print(cmd)
os.system(cmd)

level=2
filename=f"{dirname}{l1_fname}_l2.asdf"        
cmd = f"romanisim-make-image --radec {ra_cen} {dec_cen} --catalog {catalog_filename} --usecrds --webbpsf  --level {level} --bandpass F158 --date {date} --rng_seed 11 --sca {sca} {filename}"
print(cmd)
os.system(cmd)


### Run romancal on L1 data.
We now run ```romancal``` on the Level 1 data generated by ```romanisim```. To save the results the ```save_results``` argument has to be set, one can also optionally supply the output directory. The pipeline runs a series of steps one after the other. Each step can be further fine tuned by passing additional arguments via the steps keyword. The step arguments are passed on as a dictionary. In the example below we pass arguments specific to the ```source_detection``` step and the ```rampfit``` step. Note ```save_catalogs``` option has to be set in order to save the catalog. For details on steps and additional arguments see [ romancal documentation](https://roman-pipeline.readthedocs.io/en/latest/index.html).


In [ ]:
l1_fname='romanisim_mixed_sca2'
filename=f"{dirname}{l1_fname}_uncal.asdf"        
result = ExposurePipeline.call(filename,save_results=True,output_dir=dirname,
                               steps={'source_detection':
                                      {'save_catalogs':True},
                                      'rampfit':{'threshold_intercept':5.5,
                                                 'threshold_constant':0.33}})

# move catalog file to data directory
if os.path.isfile(f"{l1_fname}_uncal_tweakreg_catalog.asdf"):
    os.system(f'mv {l1_fname}_uncal_tweakreg_catalog.asdf {dirname}{l1_fname}_uncal_tweakreg_catalog.asdf')


## Open and explore the L1 and L2 images generated by romanisim 
We open the Level 1 file, its cooresponding Level 2 file and the catalog of sources identfied in the Level 2 file. We use ```roman_datamodels``` to open the first two files as it provides attribute style access to the underlying data, but one can also open them ```AsdfObject``` object for dictionary style access (see notebook on How to open Roman Data Files for more details).   

We examine the shape and the units of the data in the files. The L1 files are of shape 6x4096x4096 and have units of DN. The first dimension represents the time. The L2 files are of shape 4088x4088 and are in units of electrons/s. There is no time dimension as we have computed the slope or rate. The shape of image is smaller by 8 pixels as the 4 pixel border which has no science data has been removed. Finally, we print the sources contained in the catalog. The x and y coordinates of the centroid and the flux are shown. The centroid is in units of pixel and ranges between 0 to 4088.

In [ ]:

# read uncalibrated data
l1_fname='romanisim_mixed_sca2'
rdm1=rdm.open(f"{dirname}{l1_fname}_uncal.asdf")

# read calibrated data
rdm2=rdm.open(f"{dirname}{l1_fname}_cal.asdf")

# Read the source catalog
catalog = asdf.open(f"{dirname}{l1_fname}_uncal_tweakreg_catalog.asdf")['tweakreg_catalog'].copy()


# explore shape and units of l1 and l2 data
print(f"l1 data   shape: {str(af1['roman']['data'].shape):16} units: {af1['roman']['data'].unit}")
print(f"l2 data   shape: {str(af2['roman']['data'].shape):16} units: {af2['roman']['data'].unit}")
print()

# print the source catalog
print(f"{'index':6}  {'xcentroid':8}  {'ycentroid':8} {'flux':8}")
print("-------------------------------------")
for i in range(catalog.size):
    print(f"{i:<6}  {catalog['xcentroid'][i]:<8.2f}  {catalog['ycentroid'][i]:8.2f}  {catalog['flux'][i]:<8.2f}")


### Print information contained in the files
The information in the roman_datamodel object is stored in a hierarchical fashion and can be accessed using the attribute notation. The L1 object has three high level keys ```meta, data``` and ```amp33```, the L2 object has many more. To view the full list of available attributes and subattributes we use ```.info()``` member function. Try to locate the following atrribiutes, which we will use later, ```rdm1.meta.exposure.frame_time``` and ```rdm1.meta.exposure.read_pattern``` .     

In [ ]:
print('rdm1 keys')
print('-----------')
for key in rdm1.keys():
    print(key)
print('-----------')
print('rdm2 keys')
for key in rdm2.keys():
    print(key)
print('-----------')

print('rdm1 info')
print('-----------')
rdm1.info(max_rows=None)
print('rdm2 info')
print('-----------')
rdm2.info(max_rows=None)

### View the Level 2 image 
We now view the Level 2 images. All panels correspond to the same image but their extent are different. Each panel is a zoomed in version of the previous one.
One can see some points source objects (stars descibed by a PSF) and extended source objects (galaxies described by a sersic profile). The white spots are bad pixels and are to be ignored. The objects can be seen more clearly in the zoomed bottom left panel. One can read the coordinates and the flux values by sliding the cursor on the panels.


In [ ]:

image=rdm2.data.value

fig, axs = plt.subplots(2,2)
i,j=(4088//2,4088//2)
norm = LogNorm(vmin=2,vmax=20)
im1=axs.flat[0].imshow(image,cmap='YlOrBr_r',norm=norm,origin='lower')
im1=axs.flat[1].imshow(image,cmap='YlOrBr_r',norm=norm,origin='lower')
axs.flat[1].axis([i-1024,i+1024,j-1024,j+1024])
im1=axs.flat[2].imshow(image,cmap='YlOrBr_r',norm=norm,origin='lower')
axs.flat[2].axis([i-256,i+256,j-256,j+256])
im1=axs.flat[3].imshow(image,cmap='YlOrBr_r',norm=norm,origin='lower')
axs.flat[3].axis([i-64,i+64,j-64,j+64])
fig.colorbar(im1,ax=axs)


### View 64x64 pixel cutouts of Level 2 images centered on detected sources
We choose four sources from the source catalog and plot 64x64 pixel image cutouts centered on them. The first and fourth are point sources, while the second and third are extended sources. One can see that the point sources are tightly confined due to the PSF being very narrow and is undersampled.  

In [ ]:
# plot 64x64 cutouts of l2 images centered on detected sources
norm=LogNorm(vmin=2,vmax=100)
fig, axs = plt.subplots(2,2)
for j,i in enumerate([5, 6, 7 , 8]):
    width=32
    jp=int(catalog['xcentroid'][i])
    ip=int(catalog['ycentroid'][i])    
    im1=axs.flat[j].imshow(image[ip-width:ip+width,jp-width:jp+width],cmap='YlOrBr_r',norm=norm,origin='lower')
    axs.flat[j].set_title('Source No '+str(i))       
fig.tight_layout()
fig.colorbar(im1,ax=axs)


### Plot ramps centered on detected sources from Level 1 data and compare its slope with the value in the Level 2 data 
To plot the ramps, we being by first computing the mean time of each resultant from the read out pattern. We do crude estimate of slope by taking the difference between the first and the last resultant values and dividing it by the time difference.

A few things should be kept in mind while working with Level 1 and Level 2 data. The shape of Level 1 and Level 2 are different, as there exists a 4 pixel wide border which is ignored in Level 2. Hence, a pixel (i,j) in Level 2 corresponds to pixel (i+4,j+4) in Level 1. Also the units of Level 1 are in DN while that of Level 2 data are electron/s. The ratio of electrons to DN is know as the gain which can be found in the calibration reference file. From the printed results we see that our estimated slope is in good agreement with what is shown in Level 2 data except for a factor of 2, which is due to the gain. 

In [ ]:


frame_time=rdm1.meta.exposure.frame_time
read_pattern=rdm1.meta.exposure.read_pattern
# mean time of each resultant
t_mean=np.array([frame_time*np.mean(t) for t in read_pattern])
image1=rdm1.data.value
image2=rdm2.data.value


fig,ax=plt.subplots()
print(f"{'source no':16} {'Level 2 [i,j]':16} {'ramp slope':16} {'Level 2 data':16}")
print(f"{'':16} {'':16} {str(af1['roman']['data'].unit)+'/s':16} {af2['roman']['data'].unit:16}")
print("---------------------------------------------------------------------")
for k in [3, 4, 5, 6]:
    # image coordinates in Level 1
    ip,jp=(int(catalog['ycentroid'][k])+4,int(catalog['xcentroid'][k])+4)
    ax.plot(t_mean,image1[:,ip,jp],'-',label=f"source no: {k}, L2_image[{ip},{jp}]")
    ax.plot(t_mean,image1[:,ip,jp],'k.')
    # crude slope from Level 1
    dt=t_mean[-1]-t_mean[0]
    slope=(image1[-1,ip,jp]-image1[0,ip,jp])/dt
    print(f"{k:<16} [{ip:<4},{jp:<4}]{'':5} {slope:<16.2f} {image2[ip-4,jp-4]:<16.2f}")
ax.set_xlabel('Mean resultant time [s]')
ax.set_ylabel(f"counts [{af1['roman']['data'].unit}]")
ax.legend()



## Aditional Resources
- [romanisim](https://romanisim.readthedocs.io/en/latest/index.html)
- [romancal](https://roman-pipeline.readthedocs.io/en/latest/index.html)
- [Roman Documentation](https://roman-docs.stsci.edu)

## About this notebook
**Author:** Sanjib Sharma, Roman Telescope Branch.  
**Updated On:** 2024-23-01

***

[Top of Page](#top)
<img style="float: right;" src="https://raw.githubusercontent.com/spacetelescope/notebooks/master/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="Space Telescope Logo" width="200px"/> 